#### Import libraries

In [21]:
import torch
import sys
import os
#sys.path.append("/kaggle/input/yolov7-lib/yolov7-main")
import cv2
import numpy as np
import torch
import time
import datetime
from PIL import Image, ImageDraw, ImageFont

from models.experimental import attempt_load
from utils.general import non_max_suppression

from PIL import ImageGrab
import numpy as np
import cv2

WEIGHTS = "best.pt"
DEVICE = torch.device(0) # cpu / torch.device(0)
IMAGE_SIZE = 640
conf_threshold = 0.7
CLASSES = ['Dog','Cat','Tree']

LINE NOTIFY

In [22]:
import requests   

def notify(msg, token):
    url="https://notify-api.line.me/api/notify"     # Line Notify API 網址
    headers={"Authorization": "Bearer " + token}   # HTTPS 表頭
    payload={"message": msg}               # HTTPS 承載 (內容)
    r=requests.post(url, headers=headers, params=payload) # 提出 POST 請求
    return "訊息發送成功！"

In [23]:
def notifyImage(msg, token, image):
    url='https://notify-api.line.me/api/notify'
    headers={'Authorization': 'Bearer ' + token}
    data={'message': msg}
    image=open(image, 'rb')
    imageFile={'imageFile': image}
    r=requests.post(url, headers=headers, data=data, files=imageFile)
    if r.status_code==requests.codes.ok:
        return '圖片發送成功！'
    else:
        return f'圖片發送失敗: {r.status_code}' 

In [39]:
token='Qh118dzcCM6eyQLpo5KR4hxEalUqCfnO99QG3RGfZFu'
msg_dog = '\ndog is being naughty'
msg_cat = '\ncat is being naughty'
msg_dogncat = '\ndog & cat are being naughty'

#### Load YOLOv7

In [25]:
model = attempt_load(WEIGHTS, map_location=DEVICE)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


#### Helper

This function can be reused for object detection
It takes an image and returns keypoints as numpy array of shape ```[num_objects, 6]```.
Each object is represented as ```[x1, y1, x2, y2, conf, class_id]```.

In [26]:
def predict(image, image_size=640):
    image = np.asarray(image)
    
    # Resize image to the inference size
    ori_h, ori_w = image.shape[:2]
    image = cv2.resize(image, (image_size, image_size))
    
    # Transform image from numpy to torch format
    image_pt = torch.from_numpy(image).permute(2, 0, 1).to(DEVICE)
    image_pt = image_pt.float() / 255.0
    
    # Infer
    with torch.no_grad():
        pred = model(image_pt[None], augment=False)[0]
    
    # NMS
    pred = non_max_suppression(pred,iou_thres=0.7)[0].cpu().numpy()
    
    # Resize boxes to the original image size
    pred[:, [0, 2]] *= ori_w / image_size
    pred[:, [1, 3]] *= ori_h / image_size
    
    return pred

#### Load image

In [27]:
# IMAGE_FILE = "8.jpg"
# images = Image.open(IMAGE_FILE)
# type(images)

#### Predict

In [28]:
# pred = predict(image)

# draw = ImageDraw.Draw(image)
# for x1, y1, x2, y2, conf, class_id in pred:
#     if conf >= conf_threshold:
#         print(x1, y1, x2, y2, conf, class_id)
#         draw.rectangle(((x1, y1), (x2, y2)), width=2)
#         draw.text((x1, y1), CLASSES[int(class_id)])

In [29]:
def cross(object1,object2):
    M = [ max(object1['x1'],object2['x1']),max(object1['y1'],object2['y1']) ]
    N = [ min(object1['x2'],object2['x2']),min(object1['y2'],object2['y2']) ]
    if (N[0] > M[0]) & (N[1] > M[1]):
        return True
    else:
        return False

判斷條件式

In [33]:
def judge(image):
    pred = predict(image)
    draw = ImageDraw.Draw(image)
    tree = {'x1':0,'y1':0,'x2':0,'y2':0}
    cat = {'x1':0,'y1':0,'x2':0,'y2':0}
    dog = {'x1':0,'y1':0,'x2':0,'y2':0}
    for x1, y1, x2, y2, conf, class_id in pred:
        if conf >= conf_threshold:
            text = f"{CLASSES[int(class_id)]}  {conf:.2f}"
            font = ImageFont.truetype(r'Fruit Days.ttf', 20)
            if class_id == 2.0:
                tree = {'x1':x1,'y1':y1,'x2':x2,'y2':y2}
                draw.rectangle(((x1, y1), (x2, y2)), width=2,outline = "red")
                draw.text((x1, y1), text, fill = "red", font = font)
            elif class_id == 1.0:
                cat = {'x1':x1,'y1':y1,'x2':x2,'y2':y2}
                draw.rectangle(((x1, y1), (x2, y2)), width=2,outline = "aqua")
                draw.text((x1, y1), text, fill = "aqua", font = font)
            elif class_id == 0.0:
                dog = {'x1':x1,'y1':y1,'x2':x2,'y2':y2}
                draw.rectangle(((x1, y1), (x2, y2)), width=2,outline = "yellow")
                draw.text((x1, y1), text, fill = "yellow", font = font)
    image.save('send_test.jpg')
    try:
        if (any(dog) == True & any(cat)== True & any(tree)== True ) & (cross(dog,tree) == True) & (cross(cat,tree) == True):
            #print('dogncat')  
            return 1
        elif ( (any(cat) == True & any(tree) == True) or (any(dog) == True & any(cat)== True & any(tree)== True ) ) & (cross(cat,tree) == True):
            #print('cat')
            return 2
        elif ( (any(dog) == True & any(tree) == True) or (any(dog) == True & any(cat)== True & any(tree)== True ) ) & (cross(dog,tree) == True):
            #print('dog')
            return 3           
    except:
        return 0

In [31]:
# img = ImageGrab.grab(bbox=(0,320,1283,1033))
# frame = np.array(img)
# frame = frame[:,:,::-1]
# cv2.imshow("result",frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [38]:
result = [0]*50
sentDogNCatTime, sentDogTime, sentCatTime = 0, 0, 0
while(True):

    img = ImageGrab.grab(bbox=(0,320,1283,1033)) #bbox specifies specific region (bbox= x,y,width,height)
    #frame = np.array(img)
    judged = judge(img)
    result.append(judged)
    result.pop(0)
    
    if result.count(1) >= 40:
        nowTime = int(time.time())
        if (nowTime - sentDogNCatTime) > 10:
            #notify(msg_dogncat, token)
            notifyImage(msg_dogncat, token, 'send_test.jpg')
            sentDogNCatTime = int(time.time())
    elif result.count(2) >= 40:
        nowTime = int(time.time())
        if ((nowTime - sentCatTime) > 10) & ((nowTime - sentDogNCatTime) > 10):
            #notify(msg_cat, token)
            notifyImage(msg_cat, token, 'send_test.jpg')
            sentCatTime = int(time.time())
    elif result.count(3) >= 40:
        nowTime = int(time.time())
        if ((nowTime - sentDogTime) > 10) & ((nowTime - sentDogNCatTime) > 10):
            #notify(msg_dog, token)
            notifyImage(msg_dog, token, 'send_test.jpg')
            sentDogTime = int(time.time())



    #frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    frame = np.array(img)
    frame = frame[:,:,::-1]
    frame = cv2.resize(frame,(650,450))
    cv2.imshow("test", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    
cv2.destroyAllWindows()